# Per‑tank SAR Arc Profiles & Weekly Aggregates (Demo)

This notebook gives you two things:

1. **Visual sanity check** for the *SAR double‑bounce* feature extraction (synthetic example).
2. **Per‑tank → site aggregate** for a chosen week using the synthetic pipeline outputs (or auto‑generates them if missing).

> This runs fully offline on the scaffold — no downloads required. Replace the synthetic step with real Sentinel‑1 RTC and your tank catalog when ready.


In [ ]:
# Imports
import os, json, numpy as np, pandas as pd
import matplotlib.pyplot as plt

from src.features.sar_double_bounce import annulus_azimuth_profile, arc_features
from pathlib import Path


## 1) Synthetic SAR tank crop → azimuth profile

In [ ]:
# Create a synthetic circular tank with a bright arc near the rim
radius_px = 50
H = W = 2*radius_px + 20
cx = cy = H//2

y,x = np.indices((H,W))
dist2 = (x-cx)**2 + (y-cy)**2
img = np.random.normal(0.5, 0.10, size=(H,W)).astype("float32")
disk = dist2 <= radius_px**2
img[~disk] = 0.0

# Paint a bright arc centered at ~60°
theta = (np.degrees(np.arctan2(-(y-cy), (x-cx))) + 360) % 360
r_in = int(radius_px*0.8); r_out = int(radius_px*1.05)
arc_mask = (dist2 >= r_in**2) & (dist2 <= r_out**2) & (np.abs(((theta - 60 + 180) % 360) - 180) < 25)
img[arc_mask] += 0.5

# Compute azimuth profile and features
prof = annulus_azimuth_profile(img, cx, cy, r_px=radius_px, r_in_frac=0.7, r_out_frac=1.1, azimuth_bins=360)
feats = arc_features(prof)
feats


In [ ]:
# Plot the synthetic azimuth profile
plt.figure(figsize=(8,3))
plt.plot(prof)
plt.title("Azimuth profile (mean gamma0 by degree bin)")
plt.xlabel("Azimuth bin (deg)")
plt.ylabel("Mean")
plt.show()


## 2) Weekly per‑tank features → volume & aggregate

In [ ]:
from src.pipelines.synthetic_week import generate_synthetic_features
from src.models.calibration import apply_to_dataframe

WEEK = "2025-01-03"
features_path = Path(f"outputs/{WEEK}/per_tank_features.csv")
tanks_path = Path("data/tanks/tanks_sample.geojson")

# Auto-generate if missing (synthetic)
if not features_path.exists():
    generate_synthetic_features(str(tanks_path), str(features_path), WEEK)

df = pd.read_csv(features_path)
df.head()


In [ ]:
# Compute per-tank volume with a simple min-max calibration
g = json.loads(Path("data/tanks/tanks_sample.geojson").read_text())
geom_df = pd.DataFrame([
    {"tank_id": f["properties"].get("id", "tank_unknown"),
     "radius_m": float(f["properties"].get("radius_m", 50.0))}
    for f in g["features"]
])
df = df.merge(geom_df, on="tank_id", how="left")

df["diameter_m"] = df["radius_m"] * 2.0
df["shell_height_m"] = 18.0

df_vol = apply_to_dataframe(df, index_col="peak_to_mean", lo_col="lo", hi_col="hi",
                            diameter_m_col="diameter_m", shell_height_m_col="shell_height_m",
                            out_col="volume_bbl")
df_vol[["tank_id","volume_bbl"]]


In [ ]:
# Aggregate to a site total for the week
total_bbl = df_vol["volume_bbl"].sum()
total_bbl